# Прогнозирование продаж. Часть 2. Формирование датасета

In [ ]:
import numpy as np
import pandas as pd
import copy

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/ecommerce_data_result.csv',parse_dates=['invoicedate'])

In [ ]:
df.head()

,invoiceno,stockcode,invoicedate,customerid,country,description,amount
0,536365,21730,2010-12-01,17850,United Kingdom,GLASS STAR FROSTED T-LIGHT HOLDER,25.50
1,536365,22752,2010-12-01,17850,United Kingdom,SET 7 BABUSHKA NESTING BOXES,15.30
2,536365,71053,2010-12-01,17850,United Kingdom,WHITE METAL LANTERN,20.34
3,536365,84029E,2010-12-01,17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,20.34
4,536365,84029G,2010-12-01,17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,20.34


In [ ]:
df['customerid'].nunique()

4339

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387875 entries, 0 to 387874
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoiceno    387875 non-null  int64         
 1   stockcode    387875 non-null  object        
 2   invoicedate  387875 non-null  datetime64[ns]
 3   customerid   387875 non-null  int64         
 4   country      387875 non-null  object        
 5   description  387875 non-null  object        
 6   amount       387875 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 20.7+ MB


### Тор 7 товаров для каждого покупателя

In [ ]:
df_top_7 = df.groupby(['customerid','stockcode'])['amount'].agg(['sum','size'])

In [ ]:
df_top_7 = df_top_7.reset_index().sort_values(['customerid','sum','size'],ascending=[True,False,False])

In [ ]:
df_top_7.head(15)

,customerid,stockcode,sum,size
0,12346,23166,77183.60,1
89,12347,84558A,371.70,5
59,12347,23076,249.60,1
34,12347,22423,191.25,4
60,12347,23084,153.84,3
47,12347,22727,135.00,5
27,12347,22371,132.40,4
30,12347,22375,119.00,6
16,12347,21731,118.80,5
74,12347,23316,97.80,1


In [ ]:
df_top_7 = df_top_7.groupby(['customerid']).head(7)

In [ ]:
df_top_7.head(15)

,customerid,stockcode,sum,size
0,12346,23166,77183.60,1
89,12347,84558A,371.70,5
59,12347,23076,249.60,1
34,12347,22423,191.25,4
60,12347,23084,153.84,3
47,12347,22727,135.00,5
27,12347,22371,132.40,4
30,12347,22375,119.00,6
125,12348,POST,360.00,4
120,12348,23077,250.00,2


In [ ]:
df_top_7['rank'] = df_top_7.groupby(['customerid'])['sum'].rank(ascending=False)
df_top_7 = df_top_7.astype({'rank':int})

In [ ]:
df_top_7.head(15)

,customerid,stockcode,sum,size,rank
0,12346,23166,77183.60,1,1
89,12347,84558A,371.70,5,1
59,12347,23076,249.60,1,2
34,12347,22423,191.25,4,3
60,12347,23084,153.84,3,4
47,12347,22727,135.00,5,5
27,12347,22371,132.40,4,6
30,12347,22375,119.00,6,7
125,12348,POST,360.00,4,1
120,12348,23077,250.00,2,2


In [ ]:
df_top_7['number_rank'] = df_top_7['rank'].apply(lambda x: 'top_product_' + str(x))

In [ ]:
df_top_7.head()

,customerid,stockcode,sum,size,rank,number_rank
0,12346,23166,77183.60,1,1,top_product_1
89,12347,84558A,371.70,5,1,top_product_1
59,12347,23076,249.60,1,2,top_product_2
34,12347,22423,191.25,4,3,top_product_3
60,12347,23084,153.84,3,4,top_product_4


In [ ]:
df_top_7_pivot_table = df_top_7.pivot_table(index=['customerid'],
                                 columns=['number_rank'],
                                 values=['stockcode'],
                                 aggfunc='first').reset_index()

In [ ]:
df_top_7_pivot_table.head()

customerid     stockcode  ...                            
number_rank            top_product_1  ... top_product_6 top_product_7
0                12346         23166  ...           NaN           NaN
1                12347        84558A  ...         22371         22375
2                12348          POST  ...         84988         21213
3                12349          POST  ...         21231           NaN
4                12350          POST  ...         22348         21832

[5 rows x 8 columns]

In [ ]:
df_top_7_pivot_table.columns = df_top_7_pivot_table.columns.map(''.join)

In [ ]:
df_top_7_pivot_table.columns = [
                    'customerid',
                     'top_product_1',
                     'top_product_2',
                     'top_product_3',
                     'top_product_4',
                     'top_product_5',
                     'top_product_6',
                     'top_product_7'
                     ]

In [ ]:
df_top_7_pivot_table.head()

,customerid,top_product_1,top_product_2,top_product_3,top_product_4,top_product_5,top_product_6,top_product_7
0,12346,23166,NaN,NaN,NaN,NaN,NaN,NaN
1,12347,84558A,23076,22423,23084,22727,22371,22375
2,12348,POST,23077,23078,21211,23076,84988,21213
3,12349,POST,23253,37500,20685,NaN,21231,NaN
4,12350,POST,NaN,20615,NaN,NaN,22348,21832


In [ ]:
df_top_7_pivot_table = df_top_7_pivot_table.fillna('-')

In [ ]:
df_top_7_pivot_table.head(15)

,customerid,top_product_1,top_product_2,top_product_3,top_product_4,top_product_5,top_product_6,top_product_7
0,12346,23166,-,-,-,-,-,-
1,12347,84558A,23076,22423,23084,22727,22371,22375
2,12348,POST,23077,23078,21211,23076,84988,21213
3,12349,POST,23253,37500,20685,-,21231,-
4,12350,POST,-,20615,-,-,22348,21832
5,12352,M,POST,22423,84050,22654,22779,-
6,12353,22890,37449,37450,37446,-,-,-
7,12354,21243,-,21242,21246,21239,-,22629
8,12355,23076,23077,22649,22693,-,22423,-
9,12356,POST,22423,21843,-,22060,-,37500


### Расчет среднего чека для покупателя

In [ ]:
df_avg_check = df.groupby(['customerid']).agg({'invoiceno':['nunique'],'amount':['sum']})

In [ ]:
df_avg_check = df_avg_check.reset_index()

In [ ]:
df_avg_check.head()

,customerid,invoiceno,amount
,,nunique,sum
0,12346,1,77183.60
1,12347,7,4310.00
2,12348,4,1797.24
3,12349,1,1757.55
4,12350,1,334.40


In [ ]:
df_avg_check.columns = df_avg_check.columns.map(''.join)

In [ ]:
df_avg_check.columns = ['customerid','count_check','total_amount']

In [ ]:
df_avg_check.head()

,customerid,count_check,total_amount
0,12346,1,77183.60
1,12347,7,4310.00
2,12348,4,1797.24
3,12349,1,1757.55
4,12350,1,334.40


In [ ]:
df_avg_check['avg_check'] = df_avg_check['total_amount']/df_avg_check['count_check']
df_avg_check['avg_check'] = df_avg_check['avg_check'].apply(lambda x:round(x,2))

In [ ]:
df_avg_check.head()

,customerid,count_check,total_amount,avg_check
0,12346,1,77183.60,77183.60
1,12347,7,4310.00,615.71
2,12348,4,1797.24,449.31
3,12349,1,1757.55,1757.55
4,12350,1,334.40,334.40


### Активность покупателя в расчетный период

In [ ]:
df_number_check = copy.deepcopy(df)

In [ ]:
df_number_check.head()

,invoiceno,stockcode,invoicedate,customerid,country,description,amount
0,536365,21730,2010-12-01,17850,United Kingdom,GLASS STAR FROSTED T-LIGHT HOLDER,25.50
1,536365,22752,2010-12-01,17850,United Kingdom,SET 7 BABUSHKA NESTING BOXES,15.30
2,536365,71053,2010-12-01,17850,United Kingdom,WHITE METAL LANTERN,20.34
3,536365,84029E,2010-12-01,17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,20.34
4,536365,84029G,2010-12-01,17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,20.34


In [ ]:
df_number_check['year_month'] = df_number_check['invoicedate'].apply(lambda x:str(x.year)+'_'+str(x.month))

In [ ]:
df_number_check.head()

,invoiceno,stockcode,invoicedate,customerid,country,description,amount,year_month
0,536365,21730,2010-12-01,17850,United Kingdom,GLASS STAR FROSTED T-LIGHT HOLDER,25.50,2010_12
1,536365,22752,2010-12-01,17850,United Kingdom,SET 7 BABUSHKA NESTING BOXES,15.30,2010_12
2,536365,71053,2010-12-01,17850,United Kingdom,WHITE METAL LANTERN,20.34,2010_12
3,536365,84029E,2010-12-01,17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,20.34,2010_12
4,536365,84029G,2010-12-01,17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,20.34,2010_12


In [ ]:
df_number_check = df_number_check.groupby(['customerid','year_month'])['invoiceno'].nunique()

In [ ]:
df_number_check = df_number_check.to_frame().reset_index()

In [ ]:
df_number_check.head()

,customerid,year_month,invoiceno
0,12346,2011_1,1
1,12347,2010_12,1
2,12347,2011_1,1
3,12347,2011_10,1
4,12347,2011_12,1


In [ ]:
df_number_check_pivot_table = df_number_check.pivot_table(index=['customerid'],
                                 columns=['year_month'],
                                 values=['invoiceno'],
                                 aggfunc='sum').reset_index()

In [ ]:
df_number_check_pivot_table.columns = df_number_check_pivot_table.columns.map(''.join)

In [ ]:
df_number_check_pivot_table.columns = ['customerid', '2010_12', '2011_1', '2011_10',
       '2011_11', '2011_12', '2011_2',
       '2011_3', '2011_4', '2011_5',
       '2011_6', '2011_7', '2011_8',
       '2011_9']

In [ ]:
df_number_check_pivot_table = df_number_check_pivot_table.fillna(0)

In [ ]:
df_number_check_pivot_table.head()

,customerid,2010_12,2011_1,2011_10,2011_11,2011_12,2011_2,2011_3,2011_4,2011_5,2011_6,2011_7,2011_8,2011_9
0,12346,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12347,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,12348,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,12349,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12350,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Объединяем полученные результаты

In [ ]:
df_result = copy.deepcopy(df)

In [ ]:
df_result.head()

,invoiceno,stockcode,invoicedate,customerid,country,description,amount
0,536365,21730,2010-12-01,17850,United Kingdom,GLASS STAR FROSTED T-LIGHT HOLDER,25.50
1,536365,22752,2010-12-01,17850,United Kingdom,SET 7 BABUSHKA NESTING BOXES,15.30
2,536365,71053,2010-12-01,17850,United Kingdom,WHITE METAL LANTERN,20.34
3,536365,84029E,2010-12-01,17850,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.,20.34
4,536365,84029G,2010-12-01,17850,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE,20.34


In [ ]:
df_result = df[['customerid','country']]

In [ ]:
# У некоторые покупатели совершали операции из нескольких стран одновременно, для упрощения расчетов берем первую по списку страну
df_result = df_result.drop_duplicates(subset=['customerid'], keep='first')

In [ ]:
df_result.head()

,customerid,country
0,17850,United Kingdom
9,13047,United Kingdom
26,12583,France
46,13748,United Kingdom
65,15100,United Kingdom


In [ ]:
df_result = df_result.merge(df_avg_check, 
                            how='left',
                            left_on='customerid', right_on='customerid')

In [ ]:
df_result.tail()

,customerid,country,count_check,total_amount,avg_check
4334,13436,United Kingdom,1,196.89,196.89
4335,15520,United Kingdom,1,343.50,343.50
4336,13298,United Kingdom,1,360.00,360.00
4337,14569,United Kingdom,1,227.39,227.39
4338,12713,Germany,1,848.55,848.55


In [ ]:
df_result = df_result.drop(['count_check'], axis=1)

In [ ]:
df_result.head()

,customerid,country,total_amount,avg_check
0,17850,United Kingdom,5391.21,158.57
1,13047,United Kingdom,3237.54,323.75
2,12583,France,7281.38,485.43
3,13748,United Kingdom,948.25,189.65
4,15100,United Kingdom,876.00,292.00


In [ ]:
df_result = df_result.merge(df_top_7_pivot_table,  
                            how='left',
                            left_on='customerid', right_on='customerid')

In [ ]:
df_result.head()

,customerid,country,total_amount,avg_check,top_product_1,top_product_2,top_product_3,top_product_4,top_product_5,top_product_6,top_product_7
0,17850,United Kingdom,5391.21,158.57,22803,21730,71053,84029G,84029E,82486,22752
1,13047,United Kingdom,3237.54,323.75,84969,22960,22720,48187,47566,21755,22722
2,12583,France,7281.38,485.43,POST,22726,22727,22728,22326,22492,22556
3,13748,United Kingdom,948.25,189.65,22086,22084,22423,22950,22734,22839,23354
4,15100,United Kingdom,876.00,292.00,21258,-,-,-,-,-,-


In [ ]:
df_result = df_result.merge(df_number_check_pivot_table,  
                            how='left',
                            left_on='customerid', right_on='customerid')

In [ ]:
df_result.head()

,customerid,country,total_amount,avg_check,top_product_1,top_product_2,top_product_3,top_product_4,top_product_5,top_product_6,top_product_7,2010_12,2011_1,2011_10,2011_11,2011_12,2011_2,2011_3,2011_4,2011_5,2011_6,2011_7,2011_8,2011_9
0,17850,United Kingdom,5391.21,158.57,22803,21730,71053,84029G,84029E,82486,22752,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13047,United Kingdom,3237.54,323.75,84969,22960,22720,48187,47566,21755,22722,3.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
2,12583,France,7281.38,485.43,POST,22726,22727,22728,22326,22492,22556,1.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,2.0
3,13748,United Kingdom,948.25,189.65,22086,22084,22423,22950,22734,22839,23354,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0
4,15100,United Kingdom,876.00,292.00,21258,-,-,-,-,-,-,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4339 entries, 0 to 4338
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customerid     4339 non-null   int64  
 1   country        4339 non-null   object 
 2   total_amount   4339 non-null   float64
 3   avg_check      4339 non-null   float64
 4   top_product_1  4339 non-null   object 
 5   top_product_2  4339 non-null   object 
 6   top_product_3  4339 non-null   object 
 7   top_product_4  4339 non-null   object 
 8   top_product_5  4339 non-null   object 
 9   top_product_6  4339 non-null   object 
 10  top_product_7  4339 non-null   object 
 11  2010_12        4339 non-null   float64
 12  2011_1         4339 non-null   float64
 13  2011_10        4339 non-null   float64
 14  2011_11        4339 non-null   float64
 15  2011_12        4339 non-null   float64
 16  2011_2         4339 non-null   float64
 17  2011_3         4339 non-null   float64
 18  2011_4  

In [ ]:
df_result.to_csv('dataset_result.csv',index=False,sep=',')